# NTRK complex modeling

This notebook shows the modeling of tyrosine kinase inhibitors into NTRKs using functionalities from KinoML from the commit [968125e](https://github.com/openkinome/kinoml/commit/968125e86625102c6c718722cfeb2f0aef37f7a9). Compared to the previously used modeling pipeline, modifications in this and previous commits remove all post-translational modifications and allow for specific isoform modeling among other improvements. Furthermore, entrectinib is now included.

In [1]:
import traceback

from kinoml.core.ligands import SmilesLigand
from kinoml.core.components import BaseProtein
from kinoml.core.systems import ProteinLigandComplex
from kinoml.features.complexes import OEKLIFSKinaseHybridDockingFeaturizer

In [2]:
tkis = {'larotrectinib': 'O=C(Nc1cnn2ccc(N3CCC[C@@H]3c3cc(F)ccc3F)nc12)N1CC[C@H](O)C1',
        'selitrectinib': 'C[C@@H]1CCc2ncc(F)cc2[C@H]2CCCN2c2ccn3ncc(c3n2)C(=O)N1',
        'repotrectinib': 'C[C@H]1CNC(=O)c2cnn3ccc(nc23)N[C@H](C)c2cc(F)ccc2O1',
        'entrectinib': 'CN1CCN(c2ccc(C(=O)Nc3n[nH]c4ccc(Cc5cc(F)cc(F)c5)cc34)c(NC3CCOCC3)c2)CC1'}
ntrks = {'NTRK1': "P04629",
         'NTRK2': "Q16620",
         'NTRK3': "Q16288"}

In [3]:
docking_featurizer = OEKLIFSKinaseHybridDockingFeaturizer(loop_db="~/.OpenEye/rcsb_spruce.loop_db", shape_overlay=True)

In [4]:
for ntrk, uniprot_id in ntrks.items():
    for tki, smiles in tkis.items():
        smiles_ligand = SmilesLigand(smiles=smiles, name=tki)
        base_protein = BaseProtein(name=ntrk)
        base_protein.uniprot_id = uniprot_id
        kinase_ligand_complex = ProteinLigandComplex(components=[base_protein, smiles_ligand])
        try:
            system = docking_featurizer.featurize(kinase_ligand_complex)
        except Exception:
            print(f'Error for generating structure of {ntrk} in complex with {tki}!')
            traceback.print_exc()

Error for generating structure of NTRK2 in complex with entrectinib!


Traceback (most recent call last):
  File "<ipython-input-4-f3f63e471bd5>", line 8, in <module>
    system = docking_featurizer.featurize(kinase_ligand_complex)
  File "/home/david/github/kinoml/kinoml/features/core.py", line 37, in featurize
    features = self._featurize(system)
  File "/home/david/github/kinoml/kinoml/features/complexes.py", line 1014, in _featurize
    ligand_template["structure.ac_helix"],
  File "/home/david/github/kinoml/kinoml/features/complexes.py", line 725, in _select_kinase_structure_by_klifs_kinase_id
    f"No structure available in DFG {dfg}/alpha C helix {alpha_c_helix} conformation."
NotImplementedError: No structure available in DFG in/alpha C helix out-like conformation.


Error for generating structure of NTRK3 in complex with selitrectinib!


Traceback (most recent call last):
  File "<ipython-input-4-f3f63e471bd5>", line 8, in <module>
    system = docking_featurizer.featurize(kinase_ligand_complex)
  File "/home/david/github/kinoml/kinoml/features/core.py", line 37, in featurize
    features = self._featurize(system)
  File "/home/david/github/kinoml/kinoml/features/complexes.py", line 1014, in _featurize
    ligand_template["structure.ac_helix"],
  File "/home/david/github/kinoml/kinoml/features/complexes.py", line 725, in _select_kinase_structure_by_klifs_kinase_id
    f"No structure available in DFG {dfg}/alpha C helix {alpha_c_helix} conformation."
NotImplementedError: No structure available in DFG in/alpha C helix out conformation.


Error for generating structure of NTRK3 in complex with entrectinib!


Traceback (most recent call last):
  File "<ipython-input-4-f3f63e471bd5>", line 8, in <module>
    system = docking_featurizer.featurize(kinase_ligand_complex)
  File "/home/david/github/kinoml/kinoml/features/core.py", line 37, in featurize
    features = self._featurize(system)
  File "/home/david/github/kinoml/kinoml/features/complexes.py", line 1014, in _featurize
    ligand_template["structure.ac_helix"],
  File "/home/david/github/kinoml/kinoml/features/complexes.py", line 725, in _select_kinase_structure_by_klifs_kinase_id
    f"No structure available in DFG {dfg}/alpha C helix {alpha_c_helix} conformation."
NotImplementedError: No structure available in DFG in/alpha C helix out-like conformation.


First, NTRK inhibitor complexes were generated using the default settings (`shape_overlay=False`). In this case, ligand templates, which guide ligand placement during hybrid docking, are searched via fingerprints. This was successful for NTRK1 in complex with larotrectinib, selitrectinib and entrectinib, as well as for NTRK2 in complex with larotrectinib and selitrectinib. The other complexes were not generated and would require homology modeling or another source of kinase structures in specific conformations.

|kinase|tki          |ligand template|kinase template|kinase conformation|docking|comment                                       |
|------|-------------|---------------|---------------|-------------------|-------|----------------------------------------------|
|NTRK1 |larotrectinib|4yne           |4yne           |DFG in, aC out     |+      |                                              |
|NTRK1 |selitrectinib|4yne           |4yne           |DFG in, aC out     |+      |                                              |
|NTRK1 |repotrectinib|-              |-              |-                  |-      |                                              |
|NTRK1 |entrectinib  |5kvt           |5kvt           |DFG in, aC out-like|+      |                                              |
|NTRK2 |larotrectinib|4yne           |4at3           |DFG in, aC out     |+      |                                              |
|NTRK2 |selitrectinib|4yne           |4at3           |DFG in, aC out     |+      |amide in macrocycle rotated compared to NTRK1 |
|NTRK2 |repotrectinib|-              |-              |-                  |-      |                                              |
|NTRK2 |entrectinib  |-              |-              |-                  |-      |                                              |
|NTRK3 |larotrectinib|-              |-              |-                  |-      |                                              |
|NTRK3 |selitrectinib|-              |-              |-                  |-      |                                              |
|NTRK3 |repotrectinib|-              |-              |-                  |-      |                                              |
|NTRK3 |entrectinib  |-              |-              |-                  |-      |                                              |

Next, complexes were generated with shape_overlay set `True`. Generated complexes of NTRK1 binding larotrectinib, selitrectinib and repotrectinib match the binding modes published in [Drilon 2017](https://doi.org/10.1158/2159-8290.CD-17-0507) and show a DFG in/aC helix out conformation. Entrectinib was co-crystallized with NTRK1 and binds to the alpha C helix out-like conformation, which is not available in the PDB for NTRK2 and NTRK3. Interestingly, larotrectinib and repotrectinib are predicted to bind to DFG out conformations in case of NTRK2 and NTRK3. While the NTRK2 complexes are clashy or do not show hydrogen bonds with the hinge region, the NTRK3 complexes of larotrectinib and repotrectinib look surprisingly favorable considering the flipped binding mode compared to the NTRK1 complexes.

|kinase|tki          |ligand template|kinase template|kinase conformation|docking|comment                                               |
|------|-------------|---------------|---------------|-------------------|-------|------------------------------------------------------|
|NTRK1 |larotrectinib|4yne           |4yne           |DFG in, aC out     |+      |                                                      |
|NTRK1 |selitrectinib|4yne           |4yne           |DFG in, aC out     |+      |                                                      |
|NTRK1 |repotrectinib|4yne           |4yne           |DFG in, aC out     |+      |                                                      |
|NTRK1 |entrectinib  |5kvt           |5kvt           |DFG in, aC out-like|+      |                                                      |
|NTRK2 |larotrectinib|4ymj           |4at5           |DFG out, aC out    |+/-    |flipped compared to NTRK1, clashy with PHE of DFG     |
|NTRK2 |selitrectinib|4yne           |4at3           |DFG in, aC out     |+      |amide in macrocycle rotated compared to NTRK1         |
|NTRK2 |repotrectinib|4ymj           |4at5           |DFG out, aC out    |-      |flipped compared to NTRK1, no hbonds with hinge region|
|NTRK2 |entrectinib  |-              |-              |-                  |-      |                                                      |
|NTRK3 |larotrectinib|4ymj           |4ymj           |DFG out, aC out    |+      |flipped compared to NTRK1                             |
|NTRK3 |selitrectinib|-              |-              |-                  |-      |                                                      |
|NTRK3 |repotrectinib|4ymj           |4ymj           |DFG out, aC out    |+      |flipped compared to NTRK1                            |
|NTRK3 |entrectinib  |-              |-              |-                  |-      |                                                      |